In [4]:
import pandas
import choix
import http.client, urllib.request, urllib.parse
import itertools
from random import sample
from datetime import datetime

import http.client, urllib.request, urllib.parse
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import gspread

from functions.storage_functions import *
from functions.sleeper_functions import *


run_start = datetime.now()
print(str(run_start))

FileNotFoundError: [Errno 2] No such file or directory: 'sleeper_project.json'

In [106]:
players = update_players()

Players already up to date.


In [107]:
#Params
week_id = 2

scoring = 'ppr'
scoring = scoring.lower()

sf = True

limit = 1000

In [108]:
active = pd.read_csv('Files/active_leagues.csv')
active = active[active['scoring_type'] == scoring]
active = active[active['is_superflex'] == sf]
active = active[active['transactions'] >= 7]
active = active[active['week'] == week_id]

In [109]:
len(active)

871

In [110]:
league_list = active.league_id.to_list()

league_count = len(league_list)

if limit == None:
    league_list == league_list
elif limit > league_count:
    league_list == league_list
else:
    league_list = league_list[:limit]
    
league_count = len(league_list)
league_count

871

In [111]:
conn = http.client.HTTPSConnection('api.sleeper.app')

league_count = len(league_list)
dec = league_count * .1

lineup_t = pd.DataFrame()
errors = []

wr = pd.DataFrame()
rb = pd.DataFrame()
te = pd.DataFrame()
qb = pd.DataFrame()
defense = pd.DataFrame()
k = pd.DataFrame()
flex = pd.DataFrame()
superf = pd.DataFrame()
positions = ['WR', 'RB', 'TE', 'QB', 'FLEX']#, 'SUPER']#, 'K', 'DEF']
sup = ''

if sf == True:
    positions.remove('FLEX')
    positions.append('SUPER')
    sup = ' Super'

i = 0

In [112]:
print("Getting lineups from " + str(league_count) + " leagues.")
for league in league_list:
    i += 1
    try:
        
        conn.request("GET", '/v1/league/' + str(league) + '/matchups/' + str(week_id))
        start_conn = conn.getresponse().read()
        start = json.loads(start_conn)
        
        lineups = pd.DataFrame(start)
        
        if len(lineups) > 0:
            lineup = lineups.players.apply(pd.Series).transpose()
            lineup = lineup.melt(var_name='team_id', value_name='player_id')
            starters = lineups.starters.apply(pd.Series)
            starters = starters.melt(var_name='starter_slot', value_name='player_id')
            lineup = lineup.merge(starters, on='player_id', how='left')


            players['player_id'] = players['player_id'].astype(str)

            lineup = lineup.merge(players[['player_id','full_name','position']], on='player_id', how = 'left')
            lineup['full_name'] = np.where(lineup['position'] == 'DEF', lineup['player_id'], lineup['full_name'])
            lineup = lineup.dropna(subset=['player_id'])

            lineup['start_flag'] = np.where(lineup['starter_slot'] >= 0, 1, 0)

            teams = lineup['team_id'].drop_duplicates()


            lineup_t = lineup_t.append(lineup)

            for team in teams:

                for position in positions:

                    line = lineup[lineup['team_id'] == team]
                    #flex = line[line['position'].isin(['RB','WR','TE'])]
                    if position == "SUPER":
                        pos = line[line['position'].isin(['QB','RB','WR','TE'])]
                    elif position == "FLEX":
                        pos = line[line['position'].isin(['RB','WR','TE'])]
                    else:
                        pos = line[line['position'] == position]

                    flex_s = pos[pos['start_flag'] == 1]
                    flex_b = pos[pos['start_flag'] == 0]

                    poslist = list(itertools.product(flex_s['player_id'].astype(str), flex_b['player_id'].astype(str)))

                    df = pd.DataFrame(poslist, columns = ['starters', 'bench'])

                    if position == 'WR':
                        wr = wr.append(df).reset_index(drop=True)
                    elif position == 'RB':
                        rb = rb.append(df).reset_index(drop=True)
                    elif position == 'TE':
                        te = te.append(df).reset_index(drop=True)
                    elif position == 'QB':
                        qb = qb.append(df).reset_index(drop=True)
                    elif position == 'FLEX':
                        flex = flex.append(df).reset_index(drop=True)  
                    elif position == 'SUPER':
                        superf = superf.append(df).reset_index(drop=True)
        else:
            errors.append(league)
    except IOError as e:
        #print("skipped league " + str(league) + " " + str(i))
        errors.append(league)
        #print("I/O error({0}): {1}".format(e.errno, e.strerror))
        continue


error_df = pd.DataFrame(errors, columns=['leagues'])

print('Total Errors: ' + str(len(error_df)))

Getting lineups from 871 leagues.
Total Errors: 0


In [113]:
lineup_cnt = lineup_t.groupby('player_id').agg({'team_id':'count'}).reset_index()
lineup_cnt['ratio'] = lineup_cnt['team_id'] / league_count

In [114]:
print("Ranking players. This takes time.") 
directory = 'PairRanks/' + str(scoring) + sup + '/'
start_time = datetime.now()
d = pd.read_csv('PairRanks/data.csv')

cred = 'sleeper_project.json'
scope = ['https://spreadsheets.google.com/feeds',
     'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(cred, scope)


pb = '1WXY_aSqjzT8aRMrBGgzvTMFCI0Pa8eqSssIY1IM4xLQ'
gc = gspread.authorize(credentials)
book = gc.open_by_key(pb)


for position in positions:    
    
    pos_time = datetime.now()
    
    file = directory + position + '.csv'
    
    print("-- " + position)
    if position == 'WR':
        prefs = wr.copy()
    elif position == 'RB':
        prefs = rb.copy()
    elif position == 'TE':
        prefs = te.copy()
    elif position == 'QB':
        prefs = qb.copy()
    elif position == 'FLEX':
        prefs = flex.copy()
    elif position == 'SUPER':
        prefs = superf.copy()
    
    prefs = prefs.astype(str).merge(lineup_cnt[['player_id', 'ratio']], left_on='starters', right_on='player_id', how='left').drop_duplicates()
    prefs = prefs.astype(str).merge(lineup_cnt[['player_id', 'ratio']], left_on='bench', right_on='player_id', how='left').drop_duplicates()
    prefs = prefs[(prefs['ratio_y'] >= .2) & (prefs['ratio_x'].astype(float) >= .2)]
    prefs = prefs[['starters','bench']]
    prefs = prefs.astype(str)
    
    sl = prefs['starters'].value_counts().reset_index()
    sl.columns = ['player_id', 'starts']
    bl = prefs['bench'].value_counts().reset_index()
    bl.columns = ['player_id', 'benches']
    counts = sl.merge(bl, on='player_id', how= 'left')
    counts = counts.fillna(0)
    #counts['starts'] = np.where(counts['starts'] == np.nan, 0, counts['starts'])
    #counts['benches'] = np.where(counts['benches'] == np.nan, 0, counts['benches'])
    counts['rosters'] = counts['starts'] + counts['benches']
    counts = counts[counts['rosters'] >= dec]
    counts['diff'] = counts['starts'] - counts['benches']
    counts['diff'] = np.where(counts['diff'] == np.nan, 0, counts['diff'])
    counts['ratio'] = counts['diff'] / (counts['starts'] + counts['benches'])
    counts['score'] = counts['ratio'] * counts['starts']

    indx = prefs[['starters']]
    bnch = prefs[['bench']]
    bnch.columns = ['starters']
    indx = indx.append(bnch)
    
    indx = indx.drop_duplicates().reset_index(drop=True)
    indx['iid'] = indx.index
    indx['iid'] = indx['iid'].astype(str)
    
    
    
    prefs = prefs.merge(indx, on='starters', how='left')
    prefs = prefs.merge(indx, left_on='bench', right_on='starters', how = 'left')
    prefs.columns = ['starters', 'bench','sid', 'drop','bid']
    prefs = prefs.drop('drop', axis=1)
    prefs = prefs.dropna(subset=['bid'])
    prefs = prefs.astype(int)
    prefs = prefs.reset_index(drop=True)
    
    prefs.to_csv(directory + 'prefs.csv')
    
    comp_ids = prefs[['sid','bid']]
    
    #n_items = max(comp_ids['sid'])
    
    
    pairs = list(comp_ids.itertuples(index=False, name=None))
    
    #graph = nx.DiGraph(data=pairs)
    #nx.draw(graph, with_labels=True)
    
    n_items = len(pairs)
    #params = choix.ilsr_pairwise(n_items, pairs)
    
    

    print("---- Running model")
    test = choix.ilsr_pairwise(n_items, pairs, 0.001)
    
    print("---- Ranking...")
    ranking = np.argsort(-test)

    ranked = pd.DataFrame(ranking, columns=['sid'])
    ranked = ranked.merge(prefs[['sid', 'starters']], on = 'sid', how='left')
    ranked = ranked.drop_duplicates().reset_index(drop=True)
    ranked = ranked.dropna(subset=['starters'])
    ranked['starters'] = ranked['starters'].astype(int)
    ranked['starters'] = ranked['starters'].astype(str)
   
    names = players[['player_id', 'full_name']]
    names['player_id'] = names['player_id'].astype(str)
   
    ranked = ranked.merge(names, left_on='starters', right_on='player_id', how='left')
    ranked['rank'] = ranked.index + 1
    
    rankings = ranked[['rank','full_name']]
    rankings.columns = ['Rank', 'Player']

    counts = counts.merge(names, on = 'player_id', how='left')
    counts = counts[['player_id','full_name','starts','diff','ratio','score']]
    counts = counts.sort_values(by=['ratio', 'score'], ascending=[False,False])

    
    ranked = ranked.merge(counts, on='player_id', how='left')
    ranked = ranked[['rank','player_id','full_name_x','starts','diff','ratio','score']]
    ranked.columns = ['rank','player_id','full_name','starts','diff','ratio','score']
    
    ranked.to_csv(file, index=False)
    
    fin = str(datetime.now() - pos_time)
    sz = len(prefs)
    datum = [[scoring+sup, position, limit, sz, fin]]
    d2 = pd.DataFrame(datum, columns=['scoring','position','limit','size','time'])
    d = d.append(d2)
    
    
    
    sheet = book.worksheet(scoring.upper()+'-'+position)
    sheet.clear()
    set_with_dataframe(sheet, ranked, include_index=False)
    
    
    
    print("---- Ranked " + position + ". Time: "+ fin)
    #print(counts)

print("Finished. " + str(datetime.now() - start_time) + " | Full run: " + str(datetime.now()-run_start))
d.to_csv('PairRanks/data.csv',index=False)

Ranking players. This takes time.
-- WR
---- Running model
---- Ranking...


<ipython-input-114-c83751dcda81>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names['player_id'] = names['player_id'].astype(str)


---- Ranked WR. Time: 0:00:24.068693
-- RB
---- Running model
---- Ranking...


<ipython-input-114-c83751dcda81>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names['player_id'] = names['player_id'].astype(str)


---- Ranked RB. Time: 0:00:04.318262
-- TE
---- Running model
---- Ranking...


...

---- Ranked TE. Time: 0:00:00.205384
-- QB
---- Running model
---- Ranking...


<ipython-input-114-c83751dcda81>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names['player_id'] = names['player_id'].astype(str)


---- Ranked QB. Time: 0:00:00.518575
-- SUPER
---- Running model
---- Ranking...


<ipython-input-114-c83751dcda81>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names['player_id'] = names['player_id'].astype(str)


---- Ranked SUPER. Time: 1:43:02.695898
Finished. 1:43:36.790370 | Full run: 1:53:42.874472


#### Update Google Sheet

In [14]:
import http.client, urllib.request, urllib.parse
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import gspread

In [65]:
sheet = book.worksheet(scoring.upper()+'-'+position)
sheet.clear()
set_with_dataframe(sheet, ranked, include_index=False)

In [ ]:
cred = 'sleeper_project.json'
scope = ['https://spreadsheets.google.com/feeds',
     'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(cred, scope)


pb = '1WXY_aSqjzT8aRMrBGgzvTMFCI0Pa8eqSssIY1IM4xLQ'
gc = gspread.authorize(credentials)
book = gc.open_by_key(pb)

for position in positions:
    sheet = book.worksheet(scoring.upper()+'-'+position)
    sheet.clear()
    set_with_dataframe(sheet, df[df['active'] != 'True'], include_index=False)
    